In [1]:
import adi
import matplotlib.pyplot as plt
import pandas as pd
import neurokit2 as nk
import math
import numpy as np
import os

In [5]:
pd.set_option('display.max_columns', None) #display all columns
pd.set_option('display.max_rows', None) #display all rows

In [18]:
CHANNELS = 2
BASE_PATH_CONTROL = "../Dataset/data/control"
BASE_PATH_ALCOHOLIC = "../Dataset/data/alcoholic"
COLUMNS = ['Name', 'Age', 'Gender', 'Mean_RR', 'STD_RR', 'RMS_RR', 'Mean_HR', 'STD_HR', 'RMSSD', 'Status']

In [15]:
def extract_data(filepath):
    f = adi.read_file(filepath)
    individual_ecg = []
    for channel in range(CHANNELS):
        # for record in range(1, f.channels[channel].n_records+1):           
        record = 1
        if f.channels[channel].name == "ECG":
            data = f.channels[channel].get_data(record)
            # data = remove_artifacts(data, size=len(data))
            data = data[600:]                    
            for ele in data:
                individual_ecg.append(ele)
                        
    return individual_ecg

from get_metrics import get_rr_intervals, calc_mean_RR, calc_std_dev_RR, calc_rms_RR, calc_mean_HR, calc_std_dev_HR, calc_rmssd

In [94]:
f = open("../results/ecg.csv", 'a')
f.close()

,Name,Age,Gender,Mean_RR,STD_RR,RMS_RR,Mean_HR,STD_HR,RMSSD,Status


In [8]:
df = pd.DataFrame(columns=COLUMNS)
df

,Name,Age,Gender,Mean_RR,STD_RR,RMS_RR,Mean_HR,STD_HR,RMSSD,Status


In [19]:
base_path = BASE_PATH_ALCOHOLIC
for filename in os.listdir(f"{base_path}"):
    params = {}
    
    try:
        ecg = extract_data(f"{base_path}/{filename}")
    except:
        continue
    
    # finding R peaks
    try:
        _, rpeaks = nk.ecg_peaks(ecg, sampling_rate=1000)
        peaks = list(rpeaks['ECG_R_Peaks'])
    except ValueError:
        continue
        
    # peaks = remove_artifacts(peaks, size=len(peaks))
        
    # finding R-R Intervals
    rr_intervals = get_rr_intervals(peaks)
        
    # calculating parameters based on R-R Intervals
    params['Mean_RR'] = calc_mean_RR(rr_intervals)
    params['STD_RR'] = calc_std_dev_RR(rr_intervals)
    params['RMS_RR'] = calc_rms_RR(rr_intervals)
    params['Mean_HR'] = calc_mean_HR(rr_intervals)
    params['STD_HR'] = calc_std_dev_HR(rr_intervals)
    params['RMSSD'] = calc_rmssd(rr_intervals)

    # get patient details
    patient_details = filename.split('-')
    name = patient_details[0].replace("_", " ")
    age = patient_details[1]
    gender = patient_details[2][0]
    # date_of_ecg = patient_details[3].replace("_", " ")
    
    # status = 0 for control
    # status = 1 for alcoholic
    status = 1
    
    record = [
        name, age, gender,
        str(params['Mean_RR']),
        str(params['STD_RR']),
        str(params['RMS_RR']),
        str(params['Mean_HR']),
        str(params['STD_HR']),
        str(params['RMSSD']),
        status
    ]
    
    df.loc[len(df)] = record
    print(record)

['M Adichturali', '35', 'M', '788.9912739965096', '34.586764115114825', '789.7489947411075', '76.04646841793263', '3.402285897907258', '38.18065538220693', 1]
['Nagaraj N', '38', 'M', '646.5359712230215', '27.52688288947157', '647.121697493529', '92.80226108146904', '3.7674102981487345', '8.171400018218383', 1]
['Palaniswamy', '47', 'M', '930.395325203252', '54.99409815430027', '932.0192122439697', '64.48871611311303', '3.9862910285345436', '47.209452990672695', 1]
['Parameshwar', '26', 'M', '906.6535044422508', '55.71870349452497', '908.363996994886', '66.17743129654632', '4.018121354929002', '42.87390741183842', 1]
['Pintu', '31', 'M', '765.3784928027096', '50.96032736398178', '767.0731335472462', '78.39258688898919', '6.173785057547206', '58.31913225040885', 1]
['Prabhakaran', '57', 'M', '643.2930918846412', '16.79914322745859', '643.5124033611769', '93.27008288588854', '2.505818053386317', '17.8338528850283', 1]
['Prakash G', '43', 'M', '780.9353007945516', '46.94273218072955', '78

In [98]:
df = df.sample(frac = 1)
df

,Name,Age,Gender,Mean_RR,STD_RR,RMS_RR,Mean_HR,STD_HR,RMSSD,Status
87,Vardhaman,23,M,855.719152854512,57.21880833497826,857.6300254709595,70.11646262661262,4.555712669228255,42.99377299248151,0
93,Vincet,27,M,752.6450391315325,71.46774946496643,756.0305510645011,79.71885401547752,9.007665807514302,31.065595312519157,0
54,Ramesh,30,M,660.6515901060071,16.939869418528367,660.8687333241847,90.81942872546858,2.274696940133925,9.424568392865337,0
43,Pulak Paul,36,M,736.1759192501803,54.69546729428301,738.2049703343833,81.50225840192111,6.749842515700019,25.330473516465055,0
64,SAURABH SINGH,31,M,847.8174069089278,137.75936374367888,858.9365504837023,70.76995531237681,13.943721741445053,43.58989040799448,0
33,Pintu,31,M,712.9472459270753,40.76248077822954,714.1115846380012,84.1576993848675,5.026301184170967,32.04083186860632,0
1,Amarnath,37,M,849.8529411764706,988.2462223660465,1303.4111468170966,70.60044990482783,21.662094799243295,1221.0672554876912,0
5,Balaji,31,M,854.8155467720685,80.75048327292978,858.6211385425134,70.19058114533642,7.174997077286504,63.92662822434457,0
92,Vijendra,24,M,719.3988421694089,52.57213179932624,721.317213961106,83.40296992842643,6.496930939688799,17.21913553192874,0
117,Srinivas M.adicht,29,M,669.9209572153735,46.54918865792405,671.5362357170857,89.56280491567088,5.360577152770983,23.71195677394312,1


In [99]:
df.to_csv("../results/ecg.csv", index=False)